In [22]:
import sys
import os
sys.path.append(os.path.abspath("./MLP/MLP.py"))
sys.path.append(os.path.abspath("./datasets"))
sys.path.append(os.path.abspath("./KAN"))
from MLP import MLP
from convert_data_to_input import CreateInput
from KAN import KANWrapper
import torch

In [ ]:
# get input data
datasetPath= "./datasets/uniform_sin(x)_241121"
function_folder = "uniform_sin(x)_241121"

data = CreateInput(function_folder)

tensors = data.get_tensors()

X_train, y_train = tensors['train']

In [ ]:
# get optimal hyperparameters KAN
param_grid = {
    'kan__width': [[1, 3, 3, 1]],
    'kan__grid': [5],
    'kan__k': [3],
    'kan__seed': [42],
    'kan__lr': [0.01],
    'kan__lamb': [0.0],
    'kan__datasetPath': [datasetPath]
}
opt_params = find_best_params(datasetPath=datasetPath, param_grid=param_grid)

In [ ]:
# evaluate KAN with optimal params
KANWrapper()

In [ ]:
def create_dataset(filename: str) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    df = pd.read_csv(filename)
    x, y_noise, y_true = df['x'], df['y_noise'], df['y_true']
    return x, y_noise, y_true

In [ ]:
def evaluate_model(cfg : dict, foldername : str) -> dict:
    results = {"train_loss": [], "test_loss": 0, "x_train": [], "y_train": [], "x_test": [],
                "y_test": [], "y_train_pred": [], "y_test_pred": [], "x_plot_trainvals": [], "x_plot_testvals": [],}
    # create dataset
    train_data= create_dataset(train_file)
    test_data = create_dataset(test_file)
    val_data = create_dataset(val_file)
    true_vals = create_dataset(f"{foldername}/true_data.csv")
    # change x-values to shape (n, 1)
    x_train = torch.tensor(train_data[1].values.reshape(len(train_data[1]), 1)).float()
    y_train = torch.tensor(train_data[2].values.reshape(len(train_data[1]), 1)).float()
    x_test = torch.tensor(test_data[1].values.reshape(len(test_data[1]), 1)).float().view(-1, 1)
    y_test = torch.tensor(test_data[2].values.reshape(len(test_data[1]), 1)).float()
    x_val = torch.tensor(val_data[1].values.reshape(len(val_data[1]), 1)).float().view(-1, 1)
    y_val = torch.tensor(val_data[2].values.reshape(len(val_data[1]), 1)).float()

    x_plot_trainvals = train_data[0]
    x_plot_testvals = test_data[0]
    x_plot_valvals = val_data[0]


    # append to results
    results["x_train"] = x_train
    results["y_train"] = y_train
    results["x_test"] = x_test
    results["y_test"] = y_test
    results["x_val"] = x_val
    results["y_val"] = y_val
    results["x_plot_trainvals"] = x_plot_trainvals
    results["x_plot_testvals"] = x_plot_testvals
    results["x_plot_valvals"] = x_plot_valvals
    results["true_data_x"] = true_vals[0]
    results["true_data_ynoise"] = true_vals[1]
    results["true_data_y"] = true_vals[2]


    # train model
    model = MLP.MLP(input_size, hidden_sizes, output_size)
    # set val data
    model.set_val_data(x_val, y_val)

    start = time.time()
    model.fit(x_train,y_train,n_epochs)
    end = time.time()
    print(f"Training time for model: {model_name} was: {end - start}")
    results["training_time"] = end - start
    results["train_loss"] = model.loss_list
    results["val_loss"] = model.val_loss_list


    # test loss
    y_pred = model.predict(x_test)
    results["y_val_pred"] = model.predict(x_val)
    test_loss = model.loss_function(y_pred, y_test)
    results["test_loss"] = test_loss
    results["y_test_pred"] = y_pred
    print(f"Test loss for model: {model_name} was: {test_loss}")

    # append to results
    results["y_train_pred"] = model.predict(x_train)

    return results